In [3]:
import pandas as pd
import numpy as np
import time

np.random.seed(42)

# Подготовка данных и вспомогательные функции

def load_and_prepare_data(filepath, scaler_params=None, is_test=False):
    """Загружает данные, обрабатывает пропуски/выбросы и кодирует категориальные признаки."""
    print(f"\nОбработка данных из {filepath}")
    df = pd.read_csv(filepath)

    # Проверка наличия пропусков
    if df.isnull().sum().any():
        print("Обнаружены пропуски. Удаляем строки с пропусками.")
        df.dropna(inplace=True)
    else:
        print("Пропуски не обнаружены.")

    # Обработка выбросов bmi:
    Q1_bmi = df['bmi'].quantile(0.25)
    Q3_bmi = df['bmi'].quantile(0.75)
    IQR_bmi = Q3_bmi - Q1_bmi
    upper_bound_bmi = Q3_bmi + 1.5 * IQR_bmi
    
    # Ограничиваем выбросы сверху
    df['bmi'] = np.where(df['bmi'] > upper_bound_bmi, upper_bound_bmi, df['bmi'])

    print(f"Обработано {df.shape[0]} строк после проверки пропусков/выбросов.")

    # Приведение категориальных признаков к числовым
    categorical_features = ['sex', 'smoker', 'region']
    df = pd.get_dummies(df, columns=categorical_features, drop_first=True)
    
    # Вычисляем и печатаем полную матрицу парных корреляций
    print("\nПолная матрица парных корреляций признаков:")
    full_correlations = df.corr()
    print(full_correlations.round(2).to_string())

    # Сохраняем порядок признаков для тестового набора
    feature_names = df.drop('charges', axis=1).columns.tolist()

    # Нормализация числовых признаков (для корректной работы Градиентного Спуска)
    numerical_cols = ['age', 'bmi', 'children']
    
    if not is_test:
        # Обучение скалера только на тренировочных данных
        mean_vals = df[numerical_cols].mean().to_dict()
        std_vals = df[numerical_cols].std().to_dict()
        
        for col in numerical_cols:
            if std_vals[col] != 0:
                df[col] = (df[col] - mean_vals[col]) / std_vals[col]
            
        # Возвращаем параметры скалера для применения к тестовому набору
        scaler_params = {'mean': mean_vals, 'std': std_vals}
    else:
        # Применение скалера к тестовым данным
        if scaler_params is None:
            raise ValueError("Для тестового набора требуются параметры скалера (mean/std) с тренировочного набора.")
        
        for col in numerical_cols:
            # Применяем масштабирование
            if scaler_params['std'][col] != 0:
                df[col] = (df[col] - scaler_params['mean'][col]) / scaler_params['std'][col]
    
    # Обновляем X и y после масштабирования
    X = df.drop('charges', axis=1).values.astype(np.float64)
    y = df['charges'].values.reshape(-1, 1).astype(np.float64)

    # Добавление столбца для свободного члена
    X_b = np.c_[np.ones((X.shape[0], 1)), X]

    return X_b, y, scaler_params, feature_names


def mse(y_true, y_pred):
    """Расчет среднего квадрата ошибки."""
    return np.mean((y_true - y_pred)**2)

def predict(X, theta):
    """Расчет предсказаний."""
    return X @ theta

def calculate_gradient(X, y, theta):
    """Расчет градиента для линейной регрессии."""
    m = len(y)
    return (1/m) * X.T @ (X @ theta - y)

def calculate_regularized_gradient(X, y, theta, lambda_):
    """Расчет градиента для регрессии с регуляризацией."""
    m = len(y)
    reg_term = theta.copy()
    reg_term[0] = 0
    return (1/m) * (X.T @ (X @ theta - y) + lambda_ * reg_term)


# Многомерная линейная регрессия (аналитически и численно)

def normal_equation(X, y):
    """
    Аналитическое решение для линейной регрессии.
    Theta = (X.T * X)^-1 * X.T * y
    """
    # X.T @ X - это матрица ковариации признаков
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    return theta

def gradient_descent(X, y, alpha=0.01, iterations=10000):
    """
    Численное решение с помощью градиентного спуска.
    """
    m, n = X.shape
    theta = np.zeros((n, 1))
    
    for _ in range(iterations):
        gradient = calculate_gradient(X, y, theta)
        theta = theta - alpha * gradient
        
    return theta

# Добавление регуляризации (аналитически и численно)

def ridge_normal_equation(X, y, lambda_):
    """
    Аналитическое решение для Ridge регрессии.
    Theta = (X.T * X + lambda * I)^-1 * X.T * y
    """
    m, n = X.shape
    I = np.identity(n)
    I[0, 0] = 0 
    
    # Решение с регуляризацией
    theta = np.linalg.inv(X.T @ X + lambda_ * I) @ X.T @ y
    return theta

def regularized_gradient_descent(X, y, alpha=0.01, lambda_=1.0, iterations=10000):
    """
    Численное решение с помощью регуляризованного градиентного спуска.
    """
    m, n = X.shape
    theta = np.zeros((n, 1))
    
    for _ in range(iterations):
        gradient = calculate_regularized_gradient(X, y, theta, lambda_)
        theta = theta - alpha * gradient
        
    return theta

# Оценка обобщающей способности и выполнение

# Загрузка и подготовка тренировочных данных
X_train_b, y_train, scaler_params, feature_names = load_and_prepare_data(
    'insurance_train.csv', is_test=False
)

# Загрузка и подготовка тестовых данных, используя параметры скалера тренировочных данных
X_test_b, y_test, _, _ = load_and_prepare_data(
    'insurance_test.csv', scaler_params=scaler_params, is_test=True
)


# Настройка гиперпараметров
LEARNING_RATE = 0.01
ITERATIONS = 50000
LAMBDA = 10.0 # Параметр регуляризации для Ridge


# Константная модель (прогноз средним значением)
y_mean = np.mean(y_train)
y_pred_constant = np.full(y_test.shape, y_mean)
mse_constant = mse(y_test, y_pred_constant)


# Многомерная линейная регрессия

# Аналитическое решение
theta_analytical = normal_equation(X_train_b, y_train)
y_pred_analytical = predict(X_test_b, theta_analytical)
mse_analytical = mse(y_test, y_pred_analytical)

# Численное решение (Градиентный спуск)
theta_gd = gradient_descent(X_train_b, y_train, alpha=LEARNING_RATE, iterations=ITERATIONS)
y_pred_gd = predict(X_test_b, theta_gd)
mse_gd = mse(y_test, y_pred_gd)

# Добавление регуляризации

# Аналитическое решение с регуляризацией
theta_ridge_analytical = ridge_normal_equation(X_train_b, y_train, lambda_=LAMBDA)
y_pred_ridge_analytical = predict(X_test_b, theta_ridge_analytical)
mse_ridge_analytical = mse(y_test, y_pred_ridge_analytical)

# Численное решение с регуляризацией
theta_ridge_gd = regularized_gradient_descent(
    X_train_b, y_train, alpha=LEARNING_RATE, lambda_=LAMBDA, iterations=ITERATIONS
)
y_pred_ridge_gd = predict(X_test_b, theta_ridge_gd)
mse_ridge_gd = mse(y_test, y_pred_ridge_gd)


# Вывод результатов
print("\nОЦЕНКА ОБОБЩАЮЩЕЙ СПОСОБНОСТИ МОДЕЛЕЙ\n")

results = {
    "Константная (Прогноз средним)": mse_constant,
    "Линейная Регрессия (Аналитическая)": mse_analytical,
    "Линейная Регрессия (Градиентный Спуск)": mse_gd,
    "Ridge Регрессия (Аналитическая, Lambda=10.0)": mse_ridge_analytical,
    "Ridge Регрессия (Градиентный Спуск, Lambda=10.0)": mse_ridge_gd,
}

# Форматирование и вывод MSE
df_results = pd.DataFrame(results.items(), columns=['Модель', 'MSE'])
df_results['MSE'] = df_results['MSE'].map('{:,.2f}'.format)
print(df_results.to_string(index=False))

# Вывод весов для лучшей модели (по MSE)
best_model_name = min(results, key=results.get)
print(f"\nМодель с наименьшей ошибкой (MSE): {best_model_name}")

# Определяем веса лучшей модели
if best_model_name == "Линейная Регрессия (Аналитическая)":
    final_theta = theta_analytical
elif best_model_name == "Линейная Регрессия (Градиентный Спуск)":
    final_theta = theta_gd
elif best_model_name == "Ridge Регрессия (Аналитическая, Lambda=10.0)":
    final_theta = theta_ridge_analytical
elif best_model_name == "Ridge Регрессия (Градиентный Спуск, Lambda=10.0)":
    final_theta = theta_ridge_gd
else:
    # Константная модель не имеет весов theta, поэтому возьмем аналитическую как запасной вариант
    final_theta = theta_analytical

# Создаем таблицу весов
weights = ['Intercept'] + feature_names
weights_df = pd.DataFrame({
    'Признак': weights,
    'Вес': final_theta.flatten()
})
weights_df['Вес'] = weights_df['Вес'].map('{:,.4f}'.format)

print(f"\nНайденные Веса для лучшей из моделей:")
print(weights_df.to_string(index=False))




Обработка данных из insurance_train.csv
Пропуски не обнаружены.
Обработано 338 строк после проверки пропусков/выбросов.

Полная матрица парных корреляций признаков:
                   age   bmi  children  charges  sex_male  smoker_yes  region_northwest  region_southeast  region_southwest
age               1.00  0.08      0.05     0.30     -0.05        0.01             -0.07             -0.05              0.07
bmi               0.08  1.00      0.07     0.22      0.03        0.05             -0.19              0.35             -0.01
children          0.05  0.07      1.00     0.07     -0.01       -0.00             -0.02              0.02              0.02
charges           0.30  0.22      0.07     1.00      0.06        0.78             -0.03              0.01             -0.05
sex_male         -0.05  0.03     -0.01     0.06      1.00        0.12             -0.04              0.01             -0.04
smoker_yes        0.01  0.05     -0.00     0.78      0.12        1.00             -0.01   